# Demo notebook for Jupyter - openBIS integration
This notebook demonstrates how we can connect to the openBIS server, download some data, analyze the data and then save the results in openBIS.

## Prepare an environment

* Install packages
```bash
$ pip install --upgrade numpy matplotlib scikit-image
```


* Install jupyter-openbis extension
```bash
$ pip install --upgrade jupyter-openbis-extension
$ jupyter serverextension enable --py jupyter-openbis-server
```

When `jupyter-openbis-extension` has been installed and enabled, you can see these three extra buttons in the toolbar.
<img src="images/jupyter-openbis-extension-buttons-explained.png" width="600" height=auto />

## Configure the connection

* Click on `configure openBIS connections` button in the toolbar
* Enter 
    * Name: openbis-training
    * URL: https://openbis-training.ethz.ch/openbis/webapp/eln-lims/
    * Your username and password
    * Click `Create`
* After the status shows `connected`, choose this connection and click `Choose connection`

## Download a dataset

* Go to the lab notebook
   * Click on the dataset > `More...` > `Show Identification Info`
    * Copy the path under `Object:`, e.g., /JARUNANP_JARUNANP/TRAINING/JUPYTER-DEMO
* Go to the Jupyter notebook
    * Click on `Download openBIS datasets` button in the toolbar
    * Paste the path to `Entity Identifier/permid:`
    * Click `show datasets`
    * Select the dataset and click `Download`

## Import required modules

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.morphology import reconstruction, remove_small_objects
from skimage.measure import label, regionprops

%matplotlib inline

## Read and display the data

Before reading the image, define your dataset's code, e.g., 20201116203541033-438

In [ ]:
workdir = os.getcwd()
datasetcode = '20201116203541033-438'
path_to_dataset = os.path.join(workdir, datasetcode, 'original/blobs.tif')
path_to_dataset

In [ ]:
img = imread(path_to_dataset)

In [ ]:
fig = plt.figure(figsize=(10,5))
fig.add_subplot(121)
plt.imshow(img, cmap='gray')
plt.title('blobs')
fig.add_subplot(122)
plt.hist(img.ravel(), bins=10)
plt.title('Histogram of blobs')
plt.show()

## Process the data
Let's say our analysis task is to segment the white blobs from the dark background. We will first set a threshold to binarize the image. Then we perform some further image processing to refine the results. Finally, we label the segmented blobs and measure their size. 

In [ ]:
intensity_threshold = 150
binary = img > intensity_threshold

In [ ]:
plt.imshow(binary, cmap='gray')
plt.title('Binary image')

Fill the small holes in the white blobs.

In [ ]:
seed = np.copy(binary)
seed[1:-1, 1:-1] = binary.max()
filled = reconstruction(seed, binary, method='erosion')

In [ ]:
plt.imshow(filled, cmap='gray')
plt.title('Filled image')

Remove some small, spurious blobs.

In [ ]:
removed_small = remove_small_objects(filled.astype(bool), min_size=20)

In [ ]:
plt.imshow(removed_small, cmap='gray')
plt.title('Small objects removed')

Label the objects found.

In [ ]:
label_img = label(removed_small, connectivity=img.ndim)

In [ ]:
plt.imshow(label_img*10, cmap='prism')
plt.title('Labeled image')

Get properties of the detected objects.

In [ ]:
props = regionprops(label_img)
area = np.array([x.area for x in props])

In [ ]:
print('Found %d blobs.' % (len(area)))
print('Area min: %1.0f pixels' % (np.min(area)))
print('Area max: %1.0f pixels' % (np.max(area)))
print('Area mean: %1.0f pixels' % (np.mean(area)))

## Upload the notebook and data to openBIS

* Select `File` > `Save and checkpoint` to save your work
* Select `File` > `Download as` > `HTML` to create a static version of the notebook
* Click `upload Notebook and Data to openBIS` in the menu toolbar and fill in the dataset information
    * **choose a dataSet type**: ANALYSIS_NOTEBOOK
    * **Name**: Demo1
    * **Notes**: Results of Demo1
    * **enter a sample/experiment identifier to attach this dataSet to**: /JARUNANP_JARUNANP/TRAINING/JUPYTER-DEMO
    * uncheck all **Parent Datasets**
    * select **Files to upload**:
        * Jupyter_Demo_Notebook.html
        * Jupyter_Demo_Notebook.ipynb